reference: https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/06_CIFAR-10.ipynb

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os

# Use PrettyTensor to simplify Neural Network construction.
import prettytensor as pt

## Load the data

In [2]:
import cifar10

In [3]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [4]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


### Pre-process the data

In [5]:
from cifar10 import img_size, num_channels, num_classes
img_size_cropped = 28

In [6]:
def pre_process_image(image, training):
    # This function takes a single image as input,
    # and a boolean whether to build the training or testing graph.
    
    if training:
        # For training, add the following to the TensorFlow graph.

        # Randomly crop the input image.
        image = tf.random_crop(image, size=[img_size_cropped, img_size_cropped, num_channels])

        # Randomly flip the image horizontally.
        image = tf.image.random_flip_left_right(image)
        
        # Randomly adjust hue, contrast and saturation.
        image = tf.image.random_hue(image, max_delta=0.05)
        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)

        # Some of these functions may overflow and result in pixel
        # values beyond the [0, 1] range. It is unclear from the
        # documentation of TensorFlow 0.10.0rc0 whether this is
        # intended. A simple solution is to limit the range.

        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)
    else:
        # For training, add the following to the TensorFlow graph.

        # Crop the input image around the centre so it is the same
        # size as images that are randomly cropped during training.
        image = tf.image.resize_image_with_crop_or_pad(image,
                                                       target_height=img_size_cropped,
                                                       target_width=img_size_cropped)

    return image

In [7]:
def pre_process(images, training):
    # Use TensorFlow to loop over all the input images and call
    # the function above which takes a single image as input.
    images = tf.map_fn(lambda image: pre_process_image(image, training), images)

    return images

In [8]:
distorted_images = pre_process(images=x, training=True)

NameError: name 'x' is not defined

In [50]:
distorted_images

<tf.Tensor 'map/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, 28, 28, 3) dtype=float32>

### Get random batch

In [6]:
train_batch_size = 1000

In [7]:
def random_batch():
    # Number of images in the training-set.
    num_images = len(images_train)

    # Create a random index.
    idx = np.random.choice(num_images,
                           size=train_batch_size,
                           replace=False)

    # Use the random index to select random images and labels.
    x_batch = images_train[idx, :, :, :]
    y_batch = labels_train[idx, :]

    return x_batch, y_batch

In [8]:
x_batch.shape

NameError: name 'x_batch' is not defined

## Codes from TA's tensorflow tutorial...

In [9]:
# We start with our existing model code

def compute_logits(x):
    """Compute the logits of the model"""
    W = tf.get_variable('W', shape=[32*32*3, 10])
    b = tf.get_variable('b', shape=[10])
    
    logits = tf.add(tf.matmul(x, W), b, name='logits')
    return logits

# Note: this function is implemented in tensorflow as
# tf.nn.softmax_cross_entropy_with_logits

# We have included it here for illustration only, please don't use it.
def compute_cross_entropy(logits, y):
    y_pred = tf.nn.softmax(logits, name='y_pred') # the predicted probability for each example.
    # Compute the average cross-entropy across all the examples.
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), axis=[1]))
    return cross_entropy

def compute_accuracy(logits, y):
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy

### Tensorboard

In [18]:
images_train.shape

(50000, 32, 32, 3)

In [107]:
32*32*3

3072

In [22]:
with tf.Graph().as_default():
    # We build the model here as before
    x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')
    
    logits = compute_logits(x)
    loss = compute_cross_entropy(logits=logits, y=y)
    accuracy = compute_accuracy(logits, y)
    
    opt = tf.train.GradientDescentOptimizer(0.5)
    train_step = opt.minimize(loss)
    
    
    # Let's put the summaries below
    
    # create summary for loss and accuracy
    tf.summary.scalar('loss', loss) 
    tf.summary.scalar('accuracy', accuracy)
    
    # create summary for logits
    #tf.summary.histogram('logits', logits)
    
    # create summary for input image
    tf.summary.image('input', tf.reshape(x, [-1, 32, 32, 3]))
    
    summary_op = tf.summary.merge_all()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter('logs/example1', sess.graph)
        
        sess.run(tf.global_variables_initializer())
    
        for i in range(100):
            x_batch, y_true_batch = random_batch()
            x_batch=x_batch.reshape(5000,32*32*3)
            y_true_batch=y_true_batch.reshape(5000,10)
            feed_dict_train = {x: x_batch,y: y_true_batch}
            
            _, ac, summary = sess.run((train_step, accuracy, summary_op),
                                      feed_dict=feed_dict_train)
            # write the summary output to file
            
            summary_writer.add_summary(summary, i)

            if i % 10 == 0:
                print('Step {0}: accuracy is {1}'.format(i + 1, ac))

Step 1: accuracy is 0.09080000221729279
Step 11: accuracy is 0.10100000351667404
Step 21: accuracy is 0.10199999809265137
Step 31: accuracy is 0.10040000081062317
Step 41: accuracy is 0.09960000216960907
Step 51: accuracy is 0.09759999811649323
Step 61: accuracy is 0.10000000149011612
Step 71: accuracy is 0.10939999669790268
Step 81: accuracy is 0.09600000083446503
Step 91: accuracy is 0.09839999675750732


###  Use the Teacher's code (lec04)

In [33]:
images_train.shape

(50000, 32, 32, 3)

In [39]:
labels_train.shape

(50000, 10)

In [14]:
X_val=images_train[:5000,:,:,:]
y_onehot_val=labels_train[:5000,:]
X_train=images_train[5000:,:,:,:]
y_onehot=labels_train[5000:,:]

In [9]:
# evaluate performance on some data 
def perf_eval(logit_pred, y_true):
    """a function to evaluate performance of predicted y values vs true class labels"""
    # now look at some data
    print('    sample pred: {0}\n    sample true: {1}'.format(np.argmax(logit_pred[0:20],1),np.argmax(y_true[0:20],1)))
    # avg accuracy
    is_correct_vals = np.equal(np.argmax(logit_pred,1),np.argmax(y_true,1))
    #accuracy_vals = np.mean(is_correct_vals)
    #print('    mean classification accuracy: {0}%'.format(100*accuracy_vals))
    # Dig in a little deeper.  Where did we make correct predictions?  Does this seem reasonable?
    print('    correct predictions by class: {0}'.format(y_true[is_correct_vals,:].sum(axis=0)))

In [10]:
# cnn conv stuff
def conv(x, W):
    """simple wrapper for tf.nn.conv2d"""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool(x):
    """simple wrapper for tf.nn.max_pool with stride size 2"""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def norm(x): 
    """simple wrapper for tf.nn.lrn... See section 3.3 of Krizhevsky 2012 for details"""
    return tf.nn.lrn(x, depth_radius=5, bias=2, alpha=1e-4, beta=0.75)

In [17]:
# elaborate the compute_logits code to include a variety of models
def compute_logits(x, model_type, pkeep):
    """Compute the logits of the model"""
    if model_type=='lr':
        W = tf.get_variable('W', shape=[32*32*3, 10])
        b = tf.get_variable('b', shape=[10])
        logits = tf.add(tf.matmul(x, W), b, name='logits_lr')
    elif model_type=='cnn_cf':
        # try a 1 layer cnn
        n1 = 64
        x_image = tf.reshape(x, [-1,32,32,3]) # batch, then width, height, channels
        # cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[5, 5, 3, n1])
        b_conv1 = tf.get_variable('b_conv1', shape=[n1])
        h_conv1 = tf.nn.relu(tf.add(conv(x_image, W_conv1), b_conv1))
        # fc layer to logits
        h_conv1_flat = tf.reshape(h_conv1, [-1, 32*32*n1])
        W_fc1 = tf.get_variable('W_fc1', shape=[32*32*n1, 10])
        b_fc1 = tf.get_variable('b_fc1', shape=[10])
        logits = tf.add(tf.matmul(h_conv1_flat, W_fc1), b_fc1, name='logits_cnn_cf')
    elif model_type=='cnn_cnf':
        # try a 1 layer cnn with a normalization layer
        n1 = 64
        x_image = tf.reshape(x, [-1,32,32,3]) # batch, then width, height, channels
        # cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[5, 5, 3, n1])
        b_conv1 = tf.get_variable('b_conv1', shape=[n1])
        h_conv1 = tf.nn.relu(tf.add(conv(x_image, W_conv1), b_conv1))
        # norm layer 1
        h_norm1 = norm(h_conv1)
        # fc layer to logits
        h_flat = tf.reshape(h_norm1, [-1, 32*32*n1])
        W_fc1 = tf.get_variable('W_fc1', shape=[32*32*n1, 10])
        b_fc1 = tf.get_variable('b_fc1', shape=[10])
        logits = tf.add(tf.matmul(h_flat, W_fc1), b_fc1, name='logits_cnn_cnf')     
    elif model_type=='cnn_cpncpnff':
        # 2 layer cnn
        n1 = 32
        n2 = 64
        n3 = 1024
        x_image = tf.reshape(x, [-1,32,32,3]) # batch, then width, height, channels
        # cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[5, 5, 3, n1])
        b_conv1 = tf.get_variable('b_conv1', shape=[n1])
        h_conv1 = tf.nn.relu(tf.add(conv(x_image, W_conv1), b_conv1))
        # pool 1
        h_pool1 = maxpool(h_conv1)
        # norm 1
        h_norm1 = norm(h_pool1)
        # cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[5, 5, n1, n2])
        b_conv2 = tf.get_variable('b_conv2', shape=[n2])
        h_conv2 = tf.nn.relu(tf.add(conv(h_norm1, W_conv2), b_conv2))
        # pool 2
        h_pool2 = maxpool(h_conv2)
        # norm 2
        h_norm2 = norm(h_pool2)
        # fc layer to logits (8x8 since 2 rounds of maxpool)
        h_norm2_flat = tf.reshape(h_norm2, [-1, 8*8*n2])
        W_fc1 = tf.get_variable('W_fc1', shape=[8*8*n2, n3])
        b_fc1 = tf.get_variable('b_fc1', shape=[n3])
        h_fc1 = tf.nn.relu(tf.add(tf.matmul(h_norm2_flat, W_fc1), b_fc1))
        # one more fc layer
        # ... again, this is the logistic layer with softmax readout
        W_fc2 = tf.get_variable('W_fc2', shape=[n3,10])
        b_fc2 = tf.get_variable('b_fc2', shape=[10])
        logits = tf.add(tf.matmul(h_fc1, W_fc2), b_fc2, name='logits_cnn_cpncpnff')
    elif model_type=='cnn_cpncpnfdf':
        # same as above but add dropout.
        # 2 layer cnn
        n1 = 32
        n2 = 64
        n3 = 1024
        x_image = tf.reshape(x, [-1,32,32,3]) # batch, then width, height, channels
        # cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[5, 5, 3, n1])
        b_conv1 = tf.get_variable('b_conv1', shape=[n1])
        h_conv1 = tf.nn.relu(tf.add(conv(x_image, W_conv1), b_conv1))
        # pool 1
        h_pool1 = maxpool(h_conv1)
        # norm 1
        h_norm1 = norm(h_pool1)
        # cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[5, 5, n1, n2])
        b_conv2 = tf.get_variable('b_conv2', shape=[n2])
        h_conv2 = tf.nn.relu(tf.add(conv(h_norm1, W_conv2), b_conv2))
        # pool 2
        h_pool2 = maxpool(h_conv2)
        # norm 2
        h_norm2 = norm(h_pool2)
        # fc layer to logits (8x8 since 2 rounds of maxpool)
        h_norm2_flat = tf.reshape(h_norm2, [-1, 8*8*n2])
        W_fc1 = tf.get_variable('W_fc1', shape=[8*8*n2, n3])
        b_fc1 = tf.get_variable('b_fc1', shape=[n3])
        h_fc1 = tf.nn.relu(tf.add(tf.matmul(h_norm2_flat, W_fc1), b_fc1))
        # insert a dropout layer here.
        h_fc1_drop = tf.nn.dropout(h_fc1, pkeep)
        # one more fc layer
        # ... again, this is the logistic layer with softmax readout
        W_fc2 = tf.get_variable('W_fc2', shape=[n3,10])
        b_fc2 = tf.get_variable('b_fc2', shape=[10])
        logits = tf.add(tf.matmul(h_fc1_drop, W_fc2), b_fc2, name='logits_cnn_cpncpnfdf')
    else: 
        print('error not a valid model type')

    return logits

def compute_cross_entropy(logits, y):
    # Compute the average cross-entropy across all the examples.
    sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits, name='cross_ent_terms')
    cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

def compute_accuracy(logits, y):
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy

In [18]:
# choose case to run 
opt_method = 'sgd'
model_type = 'lr' 
dir_name = 'logs/scratch04x/'
batch_size = 100
n=50000
n_val=5000

In [15]:
batch = np.floor(np.random.rand(batch_size)*(n-n_val)).astype(int)   

In [19]:
X_batch = X_train[batch,:,:,:]..reshape([batch_size,-1])

In [21]:
X_batch.shape

(1000, 3072)

In [23]:
y_batch.shape

(1000, 32, 32, 3)

In [ ]:
with tf.Graph().as_default():
    # We build the model here as before
    x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')
    pkeep = tf.placeholder(tf.float32, name='pkeep')
    
    with tf.name_scope('model'):
        logits = compute_logits(x, model_type, pkeep)
    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y)
    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits, y)
    
    with tf.name_scope('opt'):
        if opt_method == 'sgd':
            opt = tf.train.GradientDescentOptimizer(0.5)
        elif opt_method == 'rms':
            opt = tf.train.RMSPropOptimizer(.001)
        elif opt_method == 'adam':
            opt = tf.train.AdamOptimizer(1e-4)
        train_step = opt.minimize(loss)
    
    with tf.name_scope('summaries'):
        # create summary for loss and accuracy
        tf.summary.scalar('loss', loss) 
        tf.summary.scalar('accuracy', accuracy)
        # create summary for logits
        tf.summary.histogram('logits', logits)
        # create summary for input image
        tf.summary.image('input', tf.reshape(x, [-1, 32, 32, 3]))
    
        summary_op = tf.summary.merge_all()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
        summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
        summary_writer_val = tf.summary.FileWriter(dir_name+'/val')
        
        sess.run(tf.global_variables_initializer())
    
        for i in range(20001):
            batch = np.floor(np.random.rand(batch_size)*(n-n_val)).astype(int)
            X_batch = X_train[batch,:,:,:].reshape([batch_size,-1])
            y_batch = y_onehot[batch]

            # now run
            _ , summary = sess.run((train_step, summary_op),
                                      feed_dict={x: X_batch, y: y_batch, pkeep:0.85})
            
            # write the summary output to file
            if i%100==0:
                summary_writer_train.add_summary(summary, i)

            # print diagnostics
            if i%100 == 0:
                X_batch = X_train[0:1000,:,:,:].reshape([1000,-1])
                y_batch = y_onehot[0:1000]
                (train_error,train_logits) = sess.run((accuracy,logits), {x: X_batch, y: y_batch, pkeep:1.0})
                print("\rStep {0:3d}: training accuracy {1:0.4f}".format(i, train_error), flush=True)
                # further diagnostics
                perf_eval(train_logits, y_batch)
                
            if i%100 == 0:
                X_batch = X_val.reshape([n_val,-1])
                y_batch = y_onehot_val
                (val_error, summary) = sess.run((accuracy,summary_op), {x:X_batch, y:y_batch, pkeep:1.0})
                print("\rStep {0:3d}: val accuracy {1:0.4f}".format(i, val_error), flush=True)
                summary_writer_val.add_summary(summary, i)


Step   0: training accuracy 0.1000
    sample pred: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [   0.    0.  100.    0.    0.    0.    0.    0.    0.    0.]
Step   0: val accuracy 0.1038
Step 100: training accuracy 0.1600
    sample pred: [1 1 1 1 5 1 1 1 1 1 0 1 1 1 1 0 1 5 1 1]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 40.  72.   0.   0.   0.  24.   0.   5.  19.   0.]
Step 100: val accuracy 0.1696
Step 200: training accuracy 0.1840
    sample pred: [0 0 0 1 5 1 0 1 0 1 0 1 0 1 1 0 0 2 1 0]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 99.  42.  30.   0.   0.   7.   6.   0.   0.   0.]
Step 200: val accuracy 0.1798
Step 300: training accuracy 0.2000
    sample pred: [2 0 1 5 5 5 5 5 5 5 0 5 5 1 1 2 5 5 5 5]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by cla

Step 3100: training accuracy 0.2300
    sample pred: [5 3 1 1 5 5 3 5 8 1 8 3 3 1 1 5 3 5 1 3]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [  4.  51.   0.  29.  34.  55.   0.   1.  55.   1.]
Step 3100: val accuracy 0.2598
Step 3200: training accuracy 0.2460
    sample pred: [7 9 1 1 5 6 5 5 1 1 8 9 5 1 1 5 1 5 1 6]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [  0.  67.   0.   0.  17.  62.  48.  33.   0.  19.]
Step 3200: val accuracy 0.2562
Step 3300: training accuracy 0.3380
    sample pred: [2 9 9 1 3 6 3 3 8 9 8 9 2 9 1 0 3 3 9 3]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 45.  33.  54.  31.   0.   0.  55.   0.  53.  67.]
Step 3300: val accuracy 0.3384
Step 3400: training accuracy 0.2810
    sample pred: [7 3 9 3 3 4 3 3 8 9 8 9 3 9 9 3 3 3 3 3]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class:

    sample pred: [7 2 1 1 3 4 4 4 2 7 8 7 4 1 1 4 2 4 4 4]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [  0.  22.  41.   7.  76.   4.   2.  24.  18.   6.]
Step 6100: val accuracy 0.1984
Step 6200: training accuracy 0.2610
    sample pred: [7 0 0 5 5 5 5 5 8 8 0 5 5 5 0 0 0 5 5 0]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 75.   0.   0.   4.   5.  66.  13.  47.  51.   0.]
Step 6200: val accuracy 0.2600
Step 6300: training accuracy 0.3070
    sample pred: [7 2 9 8 5 2 2 8 2 9 0 9 2 9 9 0 2 7 9 2]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 31.   2.  88.   0.   6.  14.   0.  52.  56.  58.]
Step 6300: val accuracy 0.2838
Step 6400: training accuracy 0.2530
    sample pred: [7 0 0 8 3 4 7 4 7 7 0 7 0 0 0 0 0 7 7 0]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 83.   1.  11.   2.  35.   0.   4.

Step 9200: training accuracy 0.2830
    sample pred: [7 7 9 9 3 6 7 4 7 9 0 9 7 9 9 7 3 7 9 6]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [  1.   0.  14.   9.  23.  11.  54.  98.   1.  72.]
Step 9200: val accuracy 0.2542
Step 9300: training accuracy 0.2450
    sample pred: [7 3 1 1 3 6 3 3 1 1 8 1 1 1 1 3 1 3 1 3]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [  2.  67.   2.  42.   0.   0.  59.  26.  47.   0.]
Step 9300: val accuracy 0.2584
Step 9400: training accuracy 0.3490
    sample pred: [7 9 9 9 6 4 6 4 9 9 8 9 9 9 9 4 4 7 9 6]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [  5.  10.   0.   0.  48.   3.  71.  57.  63.  92.]
Step 9400: val accuracy 0.3220
Step 9500: training accuracy 0.2320
    sample pred: [2 2 8 8 5 4 2 4 2 8 8 8 2 2 8 0 2 5 4 4]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class:

Step 12200: training accuracy 0.2940
    sample pred: [7 9 9 1 6 6 6 6 9 9 8 9 7 9 9 7 9 7 9 6]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [   0.   23.    1.    6.    0.    2.   70.   79.   11.  102.]
Step 12200: val accuracy 0.2596
Step 12300: training accuracy 0.3050
    sample pred: [7 9 9 1 3 4 3 3 9 9 8 9 3 9 1 3 3 3 9 3]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [  0.  36.   0.  68.  46.  21.   9.  47.   0.  78.]
Step 12300: val accuracy 0.2748
Step 12400: training accuracy 0.2770
    sample pred: [2 0 1 1 5 4 2 5 2 1 0 1 2 1 1 0 0 5 1 0]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 92.  43.  51.   3.  22.  38.   0.  19.   5.   4.]
Step 12400: val accuracy 0.2796
Step 12500: training accuracy 0.2120
    sample pred: [6 3 8 3 3 4 3 3 3 3 8 3 3 9 3 3 3 3 3 3]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct pred

Step 15200: training accuracy 0.2930
    sample pred: [7 0 9 1 5 5 5 5 1 9 0 9 5 9 1 0 5 5 5 5]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 65.  34.   6.   0.   1.  83.   0.  50.   1.  53.]
Step 15200: val accuracy 0.2834
Step 15300: training accuracy 0.3020
    sample pred: [2 9 9 5 5 5 5 5 2 9 0 9 5 9 9 0 2 5 9 0]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [ 42.   0.  57.   0.   0.  63.   1.   0.  51.  88.]
Step 15300: val accuracy 0.2870
Step 15400: training accuracy 0.2870
    sample pred: [2 2 9 1 5 4 2 2 2 9 8 9 2 9 1 2 2 7 9 2]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by class: [  0.  46.  90.   1.   5.  14.   0.  41.  24.  66.]
Step 15400: val accuracy 0.2556
Step 15500: training accuracy 0.2890
    sample pred: [7 0 1 1 3 2 3 3 2 7 0 7 3 1 1 0 3 7 1 3]
    sample true: [6 7 9 0 5 2 3 3 3 9 0 9 2 9 1 0 2 3 9 6]
    correct predictions by